In [5]:
# TODO : 경로를 확인하여 주세요.
%run ./MathUtil.ipynb

In [6]:
# Model 클래스의 역할 
# 목적 : AI 모델들의 동작을 지원하는 클래스이며, 자식 메서드를 통해 동작합니다. 
# 목적 : AI 모델은 MlpModel 클래스에서 정의할 예정이며, MlpModel(자식) -> Model(부모)
# 목적 : 학습, 테스트, 시각화 기능을 갖습니다. 
class Model(object):
    print("Model(object) 클래스 호출")

    # 객체 초기화 메서드 정의
    # 모델 이름, 데이터셋 정의 
    def __init__(self, model_name, dataset):
        
        # 객체 변수 생성 
        self.model_name = model_name
        self.dataset = dataset

        # [플래그 변수] 학습 여부에 대한 값을 갖는 플래그 변수 생성
        self.is_training = False 

        # hasattr() 를 활용해 표준편차 값에 대한 정의가 이뤄지지 않은 경우 디폴트 값으로 0.0030 선언 
        if not hasattr(self, "rand_std"): 
            self.rand_std = 0.0030

    
    # __str__ ()
    # 하단에 예시 코드 존재
    # 객체를 문자열로 나타내는 방법을 정의하여 보다 읽기 쉬운 형태로 출력 가능 
    ''' 
    >>> model_a_0 = Model(model_name='model_a', dataset_class='MNIST')
    >>> print(model_a_0)
    >>> [model_a/MNIST]
    '''
    def __str__(self) -> str:
        return f'[{self.model_name}/{self.dataset}]'
    
    # exec_all() 
    # 학습(train()), 테스트(test()), 시각화(visualize()) 기능을 순처적으로 동작시켜주는 메서드
    def exec_all(self, epoch_count:int = 10, batch_size:int = 10, learning_rate:float = 0.001, report:int = 3, show_cnt:int = 3) -> None:
        self.train(epoch_count, batch_size, learning_rate, report)
        self.test()  

        # 별도의 검증 데이터를 활용하여 빠르게 테스트 후 결과 시각화가 필요한 경우를 고려한 메서드 
        # visualize()의 경우 show_cnt 매개변수의 값이 0을 넘기는 경우 동작하도록 합니다. 
        # 기본값은 3 입니다. 
        if show_cnt > 0 :
            self.visualize(show_cnt)

Model(object) 클래스 호출


In [7]:
# MlpModel 클래스의 역할 
# 목적 : Model 클래스를 상속 받습니다. 
# 목적 : 학습에 필요한 전반적인 과정을 갖습니다. 
# 파라미터 초기화, 학습 시간 측정, 학습시 필요한 기능을 메서드로 갖습니다. 
# hconfigs : 은닉 계층의 노드를 리스트로 받습니다. ex) [64, 32, 10]
class MlpModel(Model):
    print("MlpModel(Model) 클래스 호출")
    
    # 상속과정에서 self.init_parameters() 메서드는 사용자가 입력한 hconfigs 값을 매개변수로 취합니다. 
    def __init__(self, model_name, dataset, hconfigs):
        super(MlpModel, self).__init__(model_name, dataset)
        self.init_parameters(hconfigs)

MlpModel(Model) 클래스 호출


In [8]:
# MlpModel.init_parameters의 역할 
# 목적 : 가중치와 편향을 초기화 하고 반환합니다. 
# shape : 가중치 생성을 위해 이전계층과 다음계층에 대한 정보를 리스트로 받습니다.
# 다음 계층에 대한 값을 [-1] 로 받아, 그 수 만큼의 편향을 생성합니다. 
def mlp_alloc_param_pair(self, shape):
    print("MlpModel.init_parameters 호출")
    weight = np.random.normal(0, self.rand_std, shape)
    
    # 가중치의 특성상 다음 계층의 노드 수 만큼 생성할 필요가 있기에 [-1] 로 리스트 마지막 값을 불러옵니다.
    bias = np.zeros([shape[-1]])

    return weight, bias

# MlpModel.alloc_layer_param 의 역할 
# 목적 : 반환된 가중치와 편향을 변수화 하여 딕셔너리 형식으로 반환합니다. 
# input_shape : 앞 계층 정보를 받습니다. 이때 첫 번째 입력 계층 정보는 튜플 형식으로 받게 됩니다. 
# hconfig : 은닉 계층 정보를 받습니다. 
def mlp_alloc_layer_param(self, input_shape, hconfig):
    print("MlpModel.alloc_layer_param 호출")

    # 입력 계층은 튜플 형식, 이후 계층에 대해서는 정수 형식으로 전달받기에
    # 두 값을 같은 형식으로 처리하기 위해 np.prod() 메서드를 활용합니다. 
    # ex) 
    # >>> np.prod((3,2))
    # 6
    # >>> np.prod((3,))
    # 3
    input_cnt = np.prod(input_shape)
    # 은닉 계층의 정보는 다음과 같이 변수 처리
    output_cnt = hconfig

    # 첫 번째 계층 정보와 두 번째 계층 정보를 받아 그에 따른 파라미터를 생성하고 반환합니다. 
    weight, bias = self.alloc_param_pair([input_cnt, output_cnt])

    # 반환 과정에서 output_cnt 를 반환해주는 이유는 다음 계층의 입력 계층의 크기 정보를 넘겨줘야 하기 때문입니다. 
    return {'w':weight, 'b':bias}, output_cnt


# MlpModel.init_parameters 의 역할
# 목표 : 은닉계층에 따른 파라미터를 변수화
# 관계 : MlpModel 클래스에서 호출 
# hconfigs : 은닉계층의 정보를 리스트로 받습니다. 
 
def mlp_init_parameters(self, hconfigs):
    print("MlpModel.init_parameters 호출")

    # 은닉 계층의 정보를 변수로 치환합니다. 
    self.hconfigs = hconfigs
    # 파라미터의 모든 정보를 담기 위한 빈 변수를 생성 
    self.pm_hiddens = []

    # Dataset 클래스에서 정의된 값을 불러와 입력 계층의 노드 수를 의미하는 변수(prev_shape)에 할당합니다. 
    prev_shape = self.dataset.input_shape

    # 은닉계층 리스트 내에 있는 값을 하나씩 불러와 은닉계층 내의 파라미터를 생성하는 역할을 수행 
    for hconfig in hconfigs:
        
        # self.alloc_layer_param 메서드의 반환값을 활용해 파라미터는 pm_hiddens 메서드에 저장하고
        # prev_shape 변수는 다시 self.alloc_layer_param 메서드에 할당하여 다음 계층의 파라미터를 생성
        pm_hidden, prev_shape = self.alloc_layer_param(prev_shape, hconfig)
        self.pm_hiddens.append(pm_hidden)

    # 위 과정을 거치면 마지막 계층 사이의 파라미터를 제외한 모든 파라미터가 생산되며,
    # 가장 마지막 계층의 파라미터를 위해 Dataset 클래스에서 정의한 output_shape 값을 활용합니다. 
    output_cnt = int(np.prod(self.dataset.output_shape))
    # 마찬가지로 은닉계층의 마지막 정보와 출력 계층의 정보를 활용하여 파라미터를 생산 합니다. 
    self.pm_output, _ = self.alloc_layer_param(prev_shape, output_cnt)
    
MlpModel.alloc_param_pair = mlp_alloc_param_pair
MlpModel.alloc_layer_param = mlp_alloc_layer_param
MlpModel.init_parameters = mlp_init_parameters

In [9]:
# MlpModel.train 의 역할
# 목적 : 학습을 진행합니다. 
# 목적 : 학습에 소요되는 시간을 측정 합니다. 
# 목적 : 평가를 진행합니다. 
# 목적 : 위 세가지를 화면에 표기 합니다. 
# epoch_count : 학습 횟수 
# batch_size : 1 에폭에 필요한 배치 
# learning_rate : 학습률
# report : 출력 주기를 설정
def mlp_model_train(self, epoch_count=10, batch_size=10, learning_rate=0.001, report=0) :
    print("MlpModel.train 호출")
    self.learning_rate = learning_rate
    
    # self.dataset.train_count는 
    # Dataset 클래스에 데코레이터를 활용한 train_count 메서드를 활용하여 
    # 학습데이터의 독립변수 값을 반환 받습니다. 
    # 이 값을 활용하여 총 배치의 수를 확보합니다. 
    batch_count = int(self.dataset.train_count / batch_size)

    # 시간측정을 위한 time,1 time2 변수를 확보 합니다. 
    time1 = time2 = int(time.time())
    # 학습 주기를 의미하는 report 변수 값이 0이 아닌 경우 학습시작을 의미하는 메시지를 출력
    if report != 0:
        print('[안내] {} AI Model 학습을 시작합니다.'.format(self.model_name))

    # 매개변수로 전달된 epoch_count 값 만큼 학습을 반복 수행합니다. 
    for epoch in range(epoch_count):

        # 손실(costs)과 정확도(accs)를 담기 위한 빈 변수를 선언
        costs = []
        accs = []

        # 1 배치에 크기(batch_size) 와 1 에폭에 필요한 전체 배치 수(batch_count)를 곱한 값을 매개변수로 하여 
        # 학습 데이터를 위한 셔플링된 인덱스를 확보합니다. 
        self.dataset.shuffle_train_data(batch_size * batch_count)
        
        # 학습 데이터의 독립변수와 종속변수를 반환하는 
        for n in range(batch_count):
            # Dataset 클래스의 get_train_data 메서드를 통해 
            # 1 배치 사이즈와 배치 순서를 매개변수로 전달해 
            # 학습 데이터에 대한 독립 종속변수를 반환받습니다. 
            trX, trY = self.dataset.get_train_data(batch_size, n)
            # Model_Mlp.ipyib 파일 하단에 정의된 train_step 메서드를 통해  
            # 배치사이즈에 맞는 학습 데이터에 대한 손실과 정확도를 반환받습니다. 
            # train_step() 에는 순전파, 역전파, 매개변수 갱신 등의 기능이 있습니다. 
            cost, acc = self.train_step(trX, trY)

            # append() 메서드를 통해 누적
            costs.append(cost)
            accs.append(acc)

        # 학습 결과를 매개변수 report 의 주기만큼 검증데이터를 활용하여 학습 성능을 확인합니다. 
        if report > 0 and (epoch+1) % report == 0:
            # Dataset 클래스 내에 검증 데이터를 반환하는 메서드를 활용하여 
            # 검증 데이터의 독립변수와 종속변수를 확보합니다. 
            vaX, vaY = self.dataset.get_validate_data(100)
            # 평가를 위한 메서드를 동작 시킵니다. 
            # 단, 이 메서드에서 직접적인 평가가 아닌, AI Mode 에 맞는 평가를 수행하기 위한 하위 메서드가 존재합니다. 
            acc = self.eval_accuracy(vaX, vaY)
            
            # 현 시점 까지의 시간을 측정 
            time3 = int(time.time())
            # 현 시점까지의 소요 시간과 전체 누적 소요시간 측정을 진행합니다. 
            tm1, tm2 = time3-time2, time3-time1

            # Dataset 클래스의 자식 클래스 Office_Image_Dataset의 train_prt_result 메서드가 동작합니다. 
            self.dataset.train_prt_result(epoch+1, costs, accs, acc, tm1, tm2)
            
            time2 = time3

    # 전체 소요시간 측정
    tm_total = int(time.time()) - time1
    # 학습 종료 메시지 출력 
    print('[안내] {} AI Model 학습을 종료합니다. 학습 소요 시간 : {} 초'.format(self.model_name, tm_total))
    
MlpModel.train = mlp_model_train

In [10]:
# MlpModel.test 의 역할 
# 목적 : 테스트 데이터 확보
# 목적 : 평가 
# 목적 : 테스트 소요시간 출력
# 목적 : 평가 결과 출력 
def mlp_model_test(self):
    print("MlpModel.test 호출")
    
    # Dataset 클래스 내의 테스트 데이터를 확보하는 메서드를 통해 테스트 데이터의 독립변수와 종속변수를 확보합니다. 
    teX, teY = self.dataset.get_test_data()
    
    time1 = int(time.time())

    # 평가를 위한 메서드이나, 즉시 평가로 진행되지 않고, ai_mode 에 따른 평가를 진행하기 위한 하위 메서드가 존재합니다. 
    acc = self.eval_accuracy(teX, teY)
    time2 = int(time.time())
    
    # 평가를 위한 출력문 메서드입니다.  
    self.dataset.test_prt_result(self.model_name, acc, time2-time1)

MlpModel.test = mlp_model_test

In [11]:
# MlpModel.visualize 의 역할 
# 목적 : 매개 변수를 중심으로 데이터를 확보합니다. 
# num : 확보하고 싶은 데이터 수를 입력 받습니다. 
def mlp_model_visualize(self, num):
    print("MlpModel.visualize 호출")
    print('[안내] {} AI Model 결과를 출력합니다.'.format(self.model_name))
    
    # 데이터 확보, 추측, 시각화 작업을 수행합니다. 
    deX, deY = self.dataset.get_visualize_data(num)
    est = self.get_estimate(deX)

    # 독립변수, 예측, 종속변수를 전달받아 추정 결과를 화면에 출력합니다. 
    self.dataset.visualize(deX, est, deY)

MlpModel.visualize = mlp_model_visualize

In [12]:
# MlpModel.train_step 의 역할 
# 목적 : 학습에 필요한 순전파 메서드 동작 이후 예측값을 확보 및 반환 합니다. 
# 목적 : 학습에 필요한 역전파 메서드 동작 이후 손실값을 확보 및 반환 합니다.
# 목적 : 학습의 주 목적인 파라미터 업데이트를 수행합니다. 
def mlp_train_step(self, x, y):
    print("MlpModel.train_step 호출")
    
    # 학습 과정을 위해 플래그 변수는 Ture 값으로 선언합니다. 
    self.is_training = True
    
    # 순전파, 손실값 반환, 평가 세 단계를 수행합니다. 
    # aux_nn 은(는) neuralnet 의 약자
    # aux_pp 은(는) postproc 의 약자
    output, aux_nn = self.forward_neuralnet(x)
    loss, aux_pp = self.forward_postproc(output, y)

    accuracy = self.eval_accuracy(x, y, output)
    
    # 역전파, 파라미터 갱신 단계를 수행합니다. 
    G_loss = 1.0 # 초기 기울기 값은 1.0 이므로, 강제로 값을 선언합니다. 

    # ∂L/∂W 혹은 ∂L/∂B 를 구하기 위해 필요한 ∂L/∂Y 편미분 연산 (변수명 : G_output)
    # 가중치 업데이트 수식 일부 ∂L/∂W = ∂L/∂Y * ∂Y/∂W
    # 편향 업데이트 수식 일부 ∂L/∂B = ∂L/∂Y * ∂Y/∂B
    G_output = self.backprop_postproc(G_loss, aux_pp)
    
    # 위에서 연산된 G_output과 보조지표를 활용해 파라미터 업데이트 과정을 수행 
    self.backprop_neuralnet(G_output, aux_nn)

    # 학습이 종료되었으므로, 학습 종료를 의미하는 플래그 변수를 조정합니다. 
    self.is_training = False

    return loss, accuracy

MlpModel.train_step = mlp_train_step

In [13]:
# MlpModel.forward_neuralnet 의 역할 
# 목적 : 순전파 역할에 따른 예측값(output)을 반환합니다. 
# x : 독립변수

def mlp_forward_neuralnet(self, x):
    print("MlpModel.forward_neuralnet 호출")
    # 독립변수와 역전파를 위해 필요한 보조지표를 담는 빈 변수를 선언합니다. 
    hidden = x
    aux_layers = []
    
    # 은닉 계층의 정보에 맞춰 순전파를 수행합니다. 
    for n, hconfig in enumerate(self.hconfigs):
        # 독립변수(hidden), 은닉 계층 정보(hconfig), 파라미터 정보(self.pm_hiddens[n])을 전달받아
        # 순전파를 위한 메서드의 매개변수에 할당합니다. 
        # 순전파 과정에서 ReLU 활성화 함수가 기본적으로 활용됩니다. 
        hidden, aux = self.forward_layer(hidden, hconfig, self.pm_hiddens[n])
        # 보조지표는 따로 aux_layers 라는 변수에 저장해둡니다. 
        # 회귀, 이진, 다중 에 따라 보조지표는 값이 달라집니다. 
        aux_layers.append(aux)

    # 마지막 은닉계층과 출력 계층의 순전파 연산을 진행합니다.
    # 마지막 계층의 순전파는 활성화 함수를 취급하지 않습니다. 
    output, aux_out = self.forward_layer(hidden, None, self.pm_output)
    
    # 순전파 결과에 따른 최종 예측값(output) 과 보조지표를 반환합니다. 
    return output, [aux_out, aux_layers]

# MlpModel.backprop_neuralnet 의 역할 
# 목적 : 역전파에 따른 파라미터 업데이트 과정 수행 
# G_output : ∂Y/YL
# aux : 역전파 과정에 필요한 변수 
def mlp_backprop_neuralnet(self, G_output, aux):
    print("MlpModel.backprop_neuralnet 호출")
    # aux 변수는 [aux_out, aux_layers] 와 같이 
    # 마지막 계층에 따른 보조지표, 이전 계층에 따른 보조지표로 구성되어 있습니다.
    aux_out, aux_layers = aux
    
    # 마지막 계층에 대한 파라미터 업데이트 과정 수행
    # 마지막 이전 계층에 대한 파라미터 업데이트를 위해 G_hidden 값을 반환 
    G_hidden = self.backprop_layer(G_y = G_output, hconfig = None, pm = self.pm_output, aux = aux_out)
    
    # 은닉 계층 단위마다 파라미터 업데이트를 위해 반복문을 수행 
    # 단, 계층마다의 은닉계층정보(self.hconfigs[n]), 파라미터(self.pm_hiddens[n]), 보조지표 (aux_layers[n]) 값을 확보하기 인덱스 확보 목적으로 reversed() 메서드를 통한 n 값을 확보
    # ex) 
    # for n in reversed(range(len([5,3,1]))):
    # print(n)
    # 2
    # 1
    # 0
    for n in reversed(range(len(self.hconfigs))):
        hconfig, pm, aux = self.hconfigs[n], self.pm_hiddens[n], aux_layers[n]
        G_hidden = self.backprop_layer(G_y = G_hidden, hconfig = hconfig, pm=pm, aux = aux)
    
    
    # return G_hidden

MlpModel.forward_neuralnet = mlp_forward_neuralnet
MlpModel.backprop_neuralnet = mlp_backprop_neuralnet

In [14]:
# MlpModel.forward_layer 의 역할 
# 목적 : 은닉 계층의 정보가 넘어오는지 파악합니다. 
# 목적 : 순전파 연산을 위 조건에 맞춰 진행 
# x : 독립변수 
# hconfig : 은닉 계층 정보 
# pm : 파라미터 값
def mlp_forward_layer(self, x, hconfig, pm):
    print("MlpModel.forward_layer 호출")
    # 순전파의 기본 연산인 선형 연산 수식을 통해 y 를 도출합니다. 
    # 파라미터는 딕셔너리 형식으로 저장되어 있기에 'w', 'b' 로 호출 합니다. 
    y = np.matmul(x, pm['w']) + pm['b']
    
    # 만약 hconfig 값이 있는경우 
    # 활성화 함수 relu() 를 반영하여 y 값을 도출합니다. 
    if hconfig is not None : 
        y = relu(y)
    
    # 입력값과 출력값은 보조지표에 활용되기에 반환받습니다.
    return y, [x,y]

# MlpModel.backprop_layer 의 역할 
# 목적 : 계층 단위 파라미터 업데이트를 수행합니다. 
# 단, 마지막 은닉계층이 아닌경우 별도의 처리가 필요함 
def mlp_backprop_layer(self, G_y, hconfig, pm, aux):
    print("MlpModel.backprop_layer 호출")
    x, y = aux
    
    # 은닉 계층 정보가 있는 경우 delta_{k+1} 를 구해줘야 하므로 이를 위한 수식을 추가 
    # 마지막 계층간에서 활용되는 경우에는 동작하지 않습니다. 
    if hconfig is not None : 
        G_y = relu_derv(y) * G_y

    # ∂L/∂W 를 구하기 위한 과정
    g_y_weight = x.transpose()
    g_y_input = pm['w'].transpose()
    
    # ∂L/∂W, ∂L/∂B 를 구하기 위한 과정
    G_weight = np.matmul(g_y_weight, G_y)
    G_bias = np.sum(G_y, axis=0)

    # 마지막 이전 계층을 위해 필요한 G_input 연산 과정 입니다. 
    # 이는 반환되고, 다시 본 메서드의 G_y 로 들어오게 됩니다. 
    G_input = np.matmul(G_y, g_y_input)
    
    # 파라미터 업데이트 과정 
    pm['w'] -= self.learning_rate * G_weight
    pm['b'] -= self.learning_rate * G_bias

    return G_input

MlpModel.forward_layer = mlp_forward_layer
MlpModel.backprop_layer = mlp_backprop_layer

In [15]:
# MlpModel.forward_postproc 의 역할
# 역할 : 손실에 대한 연산 
# 역할 : 추후 손실 과정에 대하여 수정을 고려한 메서드 및 변수 extra 를 가정합니다. 
def mlp_forward_postproc(self, output, y):
    print("MlpModel.forward_postproc 호출")
    # 순전파 연산 과정 
    loss, aux_loss = self.dataset.forward_postproc(output, y)
    extra, aux_extra = self.forward_extra_cost(y)
    return loss + extra, [aux_loss, aux_extra]

def mlp_forward_extra_cost(self, y):
    return 0, None

MlpModel.forward_postproc = mlp_forward_postproc
MlpModel.forward_extra_cost = mlp_forward_extra_cost

In [16]:
# MlpModel.backprop_postproc의 역할 
# 목적 : G_output 연산 과정 
def mlp_backprop_postproc(self, G_loss, aux):
    print("MlpModel.backprop_postproc 호출")
    aux_loss, aux_extra = aux
    self.backprop_extra_cost(G_loss, aux_extra)
    G_output = self.dataset.backprop_postproc(G_loss, aux_loss)
    return G_output

# MlpModel.backprop_extra_cost의 역할 
# 목적 : 지금은 빈 메서드 입니다. 
def mlp_backprop_extra_cost(self, G_loss, aux):
    pass

MlpModel.backprop_postproc = mlp_backprop_postproc
MlpModel.backprop_extra_cost = mlp_backprop_extra_cost

In [17]:
# MlpModel.eval_accuracy 의 역할 
# 목적 : 평가를 수행합니다. 
# 목적 : 별도의 output 이 존재하는 경우 해당 output 을 기반으로 정확도 연산 
def mlp_eval_accuracy(self, x, y, output=None):
    print("MlpModel.eval_accuracy 호출")
    if output is None:
        output, _ = self.forward_neuralnet(x)
        
    accuracy = self.dataset.eval_accuracy(x, y, output)
    return accuracy

MlpModel.eval_accuracy = mlp_eval_accuracy

In [18]:
# MlpModel.get_estimate 의 역할 
# 목적 : 예측값 연산을 수행합니다. 
def mlp_get_estimate(self, x):
    print("MlpModel.get_estimate 호출")
    output, _ = self.forward_neuralnet(x)
    estimate = self.dataset.get_estimate(output)
    return estimate

MlpModel.get_estimate = mlp_get_estimate